## Yatzy program

## Initial planning

Yatzy program to play with and get policy and probabilities

Should have
* encoding state
* transition function for rules
* checking terminal state and computing reward
* actions to roll dice and select categories
* value iteration to compute value function / policy

I want to create a yatzy program that I've stated above. Am I missing anything?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Great project! Your list covers the key components for a Yatzy reinforcement learning setup. Let me check if you've got everything:

**What you have:**
- State encoding ✓
- Transition function (rules) ✓
- Terminal state and reward ✓
- Actions ✓

**You might also want to consider:**
- A **scoring function** for each category (ones, twos, full house, etc.)
- How to handle the **scorecard state** (which categories have been used)
- The **reroll mechanism** (you get up to 3 rolls per turn, choosing which dice to keep)
- Whether you're implementing **single-player** (maximize your score) or **multi-player** (beat opponents)

For value iteration specifically, you'll need a **value function** that estimates the expected score from any state.

What's your vision for the game? Single-player trying to maximize score, or competitive play? And do you already have a sense of how you want to encode the state?

I think I could start with a single player setting. What if I want to get probability of each outcome? Should I use markov chains for that or can I use same setup I'm designing to easily get that?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Good question! You can actually use the same setup for both.

Your value iteration framework naturally gives you **expected values** (average scores), but if you want the full **probability distribution** of outcomes, you have a couple of options:

1. **Extend your existing transition function** - Since you're already building transitions (state → action → next state), you can track probabilities through these transitions. This is essentially using the Markov chain structure you're already creating.

2. **Monte Carlo simulation** - Once you have your optimal policy, you can run many simulations and collect the distribution of final scores empirically.

The first approach is more exact but computationally intensive. The second is simpler to implement and often sufficient.

Which sounds more appealing to you? And have you thought about how you want to represent your state yet (dice values, roll number, which categories are filled)?

State should have
* current dice values
* roll number, or rolls left, same thing I guess
* which categories are filled

do you think thats sufficient for state?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Yes, that's sufficient for the state! You've captured all the information needed to make optimal decisions:

- **Current dice values** - what you're working with right now
- **Rolls left** - determines whether you can reroll or must score
- **Filled categories** - which scoring options are still available

One small implementation detail to consider: how will you represent the dice? For example, `[1, 2, 3, 3, 5]` vs. a sorted version `[1, 2, 3, 3, 5]` vs. counts `{1:1, 2:1, 3:2, 5:1}`. Sorting or using counts can reduce your state space significantly since `[1, 2, 3, 3, 5]` and `[3, 1, 5, 3, 2]` are functionally identical.

Have you thought about how you want to represent the dice values?

Sorted version is best. Rolls left can just be 2,1,0. Hmm categories filled. What options I have, a binary number or list?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Good choices! Sorted dice and rolls left as `2, 1, 0` are clean.

For categories filled, both options work:

**Binary number** (e.g., `0b1010110101`) - compact, efficient for hashing/lookup, but less readable

**List/set** (e.g., `['ones', 'full_house']` or `{0, 3, 5}`) - more readable, easier to debug

**Tuple of booleans** (e.g., `(True, False, True, ...)`) - middle ground, hashable for dictionaries

The binary number is probably best for value iteration since you'll have lots of states to store. You can use bit operations to check/set categories quickly.

How many categories are you including? Standard Yatzy has 15 (6 upper section + 9 lower section). Are you doing full rules or a simplified version?

Full rules. Should I use numpy for storing state, it has bit operations also, right?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Yes, numpy has bitwise operations (`&`, `|`, `^`, `~`, `<<`, `>>`), but for storing state in value iteration, you might not need numpy arrays specifically.

Here's why: your state is more like a **key** for looking up values in a dictionary, not a large array of numbers. Python's built-in `int` for the binary categories works great with bitwise ops, and you can use a regular dict or numpy array indexed by state.

For example:
- Categories as `int`: `categories = 0b0001010` 
- Check if category 2 is filled: `categories & (1 << 2)`
- Mark category 2 as filled: `categories | (1 << 2)`

You might use numpy for dice operations (rolling, sorting) or storing your value function as an array if you enumerate all states.

Are you planning to enumerate all possible states upfront, or build them dynamically as you explore?

Enumerate up front. I think its common to represent states with int, like in gymnasium discrete? Do you think its better to use dict with state as key?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Good point! Yes, in Gymnasium's discrete spaces, states are typically integers (0, 1, 2, ..., n). This works well when you can map your full state to a unique integer.

For your case, you could create a mapping function:
- Dice (sorted) → some integer
- Rolls left (0-2) → multiply by some factor
- Categories (binary) → shift and add

Then use a numpy array indexed by this state integer for your value function: `V[state_int]`.

**Tradeoff:**
- **Int indexing + numpy array**: faster lookup, more cache-friendly
- **Dict with tuple key**: easier to debug (you can see `(dice, rolls, categories)` directly), simpler mapping

Since you're enumerating upfront and want performance for value iteration, the integer state indexing is probably better.

Do you want to start sketching out how to encode your state as an integer, or would you like to begin with a simpler piece first (like the dice rolling or scoring functions)?

Actually, when I think about it now having 
```
state = (dice, rolls, categories)
value = V[state]
```
looks very clean. That's the dict approach right?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Exactly! That's the dict approach:

```python
V = {}  # or defaultdict
state = (tuple(sorted_dice), rolls_left, categories_int)
value = V[state]
```

The tuple is hashable (so it works as a dict key), and you're right—it's very clean and readable. You can directly see what each state represents when debugging.

The performance difference between dict and array indexing probably won't matter much unless you're doing millions of iterations. And dict has a nice bonus: you only store states you actually visit, which could save memory.

Sounds like you have a good plan! What would you like to tackle first—the state encoding, the scoring functions, or the transition/action logic?

Can you give me the readable options for categories that would work well with the dict approach?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Sure! Here are some readable options for categories:

**Option 1: Named tuple or dict keys**
```python
categories = {'ones': True, 'twos': False, 'threes': True, ...}
```

**Option 2: Set of used categories**
```python
categories = {'ones', 'full_house', 'chance'}  # just the filled ones
```

**Option 3: Frozenset (hashable set)**
```python
categories = frozenset(['ones', 'full_house'])  # can be dict key
```

For your dict state approach, **Option 3 (frozenset)** works best because:
- It's hashable (required for dict keys)
- You only store what's used (compact)
- Easy to check: `'ones' in categories`
- Easy to add: `categories | {'twos'}`

Would you like to use frozenset, or do you prefer keeping the binary int approach for compactness?

What are the differences between options 1 and 3?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Good question! The key difference is **hashability**:

**Option 1 (dict):**
- `{'ones': True, 'twos': False, ...}` is **not hashable**
- Can't use it as a dict key: `V[state]` would fail
- You'd need to convert it (e.g., to a frozenset or tuple of items)

**Option 3 (frozenset):**
- `frozenset(['ones', 'full_house'])` **is hashable**
- Works directly as dict key: `V[(dice, rolls, categories)]` ✓
- Immutable (can't accidentally modify it)

So Option 1 looks nice but doesn't work for your `state` tuple without extra conversion. Option 3 works out of the box.

Does that clarify the tradeoff?

Another option would be to have an efficient way to store state and then have a function to get the human readable version.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

That's a smart approach! You get the best of both worlds:

**Efficient storage:** Use int for categories (or even encode the whole state as one int)

**Readable debugging:** Have a helper function like `state_to_readable(state)` that converts it when you need to inspect it

This is actually a common pattern in RL implementations. You could have:
- `encode_state(dice, rolls, categories)` → compact representation
- `decode_state(state)` → human-readable format

The compact version stays in your dict keys and value iteration, but you can always decode when debugging or analyzing results.

Do you want to go with this approach? And if so, would you keep categories as int and just decode those, or encode the entire state into a single int?

Is it easy to implement later? What would be a good plan to start iterating and experimenting the quickest?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Great pragmatic question! Here's the quickest path to start experimenting:

**Start simple, optimize later:**

1. Use the readable approach first: `state = (tuple(dice), rolls_left, frozenset(categories))`
2. Get your basic game logic working (scoring, transitions, actions)
3. Test with small examples
4. If performance becomes an issue, add encoding functions later

This way you can:
- Debug easily (see exactly what's happening)
- Iterate quickly on game logic
- Add the int encoding wrapper later without changing core logic

The encoding functions are straightforward to add once everything works. Premature optimization can slow you down.

Does starting with the readable version and refactoring later sound good? Or do you have performance concerns from the start?

What are advantages of frozenset vs tuple or strings?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Good question! Here are the key differences:

**Frozenset:**
- Unordered: `frozenset(['ones', 'twos'])` == `frozenset(['twos', 'ones'])`
- Easy membership check: `'ones' in categories`
- Natural for "which categories are filled" (order doesn't matter)

**Tuple:**
- Ordered: `('ones', 'twos')` ≠ `('twos', 'ones')`
- You'd need to keep it sorted to avoid duplicate states
- Still hashable and works as dict key

**Strings:**
- Could do something like `'ones,twos,full_house'`
- Harder to check membership or add categories
- More manual string manipulation

For your use case (tracking which categories are used), **frozenset** is most natural since the order you filled categories doesn't matter for game state.

Does that help you decide which to use?

## Summary of planning

Okay can you summarise all the decisions I made and have a clear plan to start working next

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

**Your Decisions:**

1. **Single-player** Yatzy to maximize score
2. **State representation:** `(tuple(sorted_dice), rolls_left, frozenset(filled_categories))`
3. **Use dict** for value function: `V[state] = value`
4. **Start with readable format**, optimize encoding later if needed
5. **Full Yatzy rules** (15 categories)

**Plan to Start:**

1. Define the 15 category names (as strings for frozenset)
2. Write scoring functions for each category
3. Implement dice rolling and state transitions
4. Define valid actions (which dice to keep, which category to score)
5. Build value iteration algorithm
6. Test and iterate

**Immediate next step:** Would you like to start by defining the category names and writing a few scoring functions (like "ones", "full house", "yatzy")? Or would you prefer to begin with the dice rolling mechanics?

In [ ]:
# categories

categories = set([
    "ones",
    "twos",
    "threes",
    "fours",
    "fives",
    "sixes",
    "one_pair",
    "two_pairs",
    "three_same",
    "four_same",
    "small_straight",
    "large_straight",
    "full_house",
    "chance",
    "yatzy",
])
categories

{'chance',
 'fives',
 'four_same',
 'fours',
 'full_house',
 'large_straight',
 'one_pair',
 'ones',
 'sixes',
 'small_straight',
 'three_same',
 'threes',
 'two_pairs',
 'twos',
 'yatzy'}

In [ ]:
# dice values
dice_vals = [5, 4, 3, 2, 1]
sorted_vals = tuple(sorted(dice_vals))
sorted_vals

(1, 2, 3, 4, 5)

In [ ]:
score = sum(sorted_vals)
score

15

In [ ]:
import random

random.choices([1,2,3,4,5,6], k=5)

[3, 1, 5, 6, 3]

In [ ]:
def roll_n_dice(n):
    return random.choices([1,2,3,4,5,6], k=n)

roll_n_dice(5)

[4, 6, 5, 2, 4]

In [ ]:
state = (sorted_vals, 2, frozenset(random.sample(sorted(categories), 3)))
state

((1, 2, 3, 4, 5), 2, frozenset({'fives', 'full_house', 'threes'}))

## Planning more about how to represent state and actions

Okay, thinking about actions.

I would have 2 rolls left.

I can choose what dices to hold and which to re-roll. So basically 5 long binary word.

I can also choose small straight category, but no chance as its taken already.

I can also choose to set zero to any category that I've not taken already.

so action could be like
```
action = [00000, "small_straight", None]
# or
action = [11000, None, None]
# or
action = [00000, None, "ones"]
```

I need to have a function to compute possible actions.
* allow re-roll if "rolls left" is > 0
* check which categories can be selected
* which categories could be set to zero

What do you think about encoding actions? What other options I could have?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Good thinking on the action structure! Your approach captures the three types of actions. Here are some encoding options:

**Option 1: Your tuple approach**
```python
action = (reroll_mask, score_category, zero_category)
```
Simple, but only one field should be non-None at a time.

**Option 2: Tagged tuple**
```python
action = ('reroll', [1,1,0,0,0])  # or
action = ('score', 'small_straight')  # or
action = ('zero', 'ones')
```
More explicit about action type.

**Option 3: Single value with different types**
```python
action = [1,1,0,0,0]  # reroll (list/tuple)
action = 'small_straight'  # score (string)
action = ('zero', 'ones')  # zero out (tuple)
```
Python's dynamic typing lets you distinguish by type.

**Option 4: Integer encoding**
Map all possible actions to integers (0-N), like Gymnasium discrete actions.

Which feels most natural to you? Option 2 is probably clearest for debugging, while Option 4 would be most efficient if you need it later.

Are there other options similar to tagged tuple?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Yes! Here are some similar options to tagged tuples:

**Named tuple:**
```python
from collections import namedtuple
Action = namedtuple('Action', ['type', 'value'])
action = Action('reroll', [1,1,0,0,0])
action = Action('score', 'small_straight')
```

**Dataclass:**
```python
from dataclasses import dataclass
@dataclass(frozen=True)  # frozen makes it hashable
class Action:
    type: str
    value: any
```

**Dictionary:**
```python
action = {'type': 'reroll', 'value': [1,1,0,0,0]}
action = {'type': 'score', 'category': 'small_straight'}
```
(Not hashable unless you convert it)

**Enum + value:**
```python
from enum import Enum
class ActionType(Enum):
    REROLL = 1
    SCORE = 2
    ZERO = 3
action = (ActionType.REROLL, [1,1,0,0,0])
```

Named tuple and dataclass give you nice attribute access (`action.type`, `action.value`) while staying hashable. Do any of these appeal to you more than the plain tagged tuple?

## Back to implementing

In [ ]:
from dataclasses import dataclass

@dataclass
class State:
    dice_values: tuple[int, ...]
    rolls_left: int
    categories_picked: frozenset[str]

state = State((1, 2, 3, 4, 5), 2, frozenset({'chance', 'large_straight', 'yatzy'}))
state

State(dice_values=(1, 2, 3, 4, 5), rolls_left=2, categories_picked=frozenset({'large_straight', 'chance', 'yatzy'}))

In [ ]:
from typing import Literal

@dataclass
class Action:
    type: Literal["reroll", "score", "zero"]
    value: tuple[int, ...] | str

Action("reroll", (0,0,1,0,0))

Action(type='reroll', value=(0, 0, 1, 0, 0))

In [ ]:
import itertools

reroll_masks = list(itertools.combinations_with_replacement((0,1), 5))
reroll_masks

[(0, 0, 0, 0, 0),
 (0, 0, 0, 0, 1),
 (0, 0, 0, 1, 1),
 (0, 0, 1, 1, 1),
 (0, 1, 1, 1, 1),
 (1, 1, 1, 1, 1)]

In [ ]:
available_actions = []
if state.rolls_left > 0:
    rerolls = [Action("reroll", mask) for mask in reroll_masks]
    available_actions.extend(rerolls)
available_actions

[Action(type='reroll', value=(0, 0, 0, 0, 0)),
 Action(type='reroll', value=(0, 0, 0, 0, 1)),
 Action(type='reroll', value=(0, 0, 0, 1, 1)),
 Action(type='reroll', value=(0, 0, 1, 1, 1)),
 Action(type='reroll', value=(0, 1, 1, 1, 1)),
 Action(type='reroll', value=(1, 1, 1, 1, 1))]

In [ ]:
can_zero = categories - state.categories_picked
can_zero

{'fives',
 'four_same',
 'fours',
 'full_house',
 'one_pair',
 'ones',
 'sixes',
 'small_straight',
 'three_same',
 'threes',
 'two_pairs',
 'twos'}

I need checker functions for
* has_ones, has_twos, has_threes, ... I think I could have has_num(n) or something
* has_pair
* has_two_pairs
* has_three_same
* has_four_same
* has_small_straight
* has_large_straight
* has_full_house
* has_yatzy

Can you summarise where I'm at, whats still missing?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

**Where you're at:**

* ✅ State representation (dataclass with dice, rolls_left, categories_picked)
* ✅ Action representation (dataclass with type and value)
* ✅ Category names defined
* ✅ Basic dice rolling function
* ✅ Started on available actions (reroll actions working)

**Still missing:**

1. **Checker functions** - detect patterns in dice (pairs, straights, full house, etc.)
2. **Scoring functions** - calculate points for each category given dice
3. **Complete available actions** - add "score" and "zero" actions based on what's valid
4. **Transition function** - apply action to state and get next state
5. **Reward/terminal logic** - when game ends and final score calculation
6. **Value iteration** - the RL algorithm itself

**Immediate next step:** The checker functions you listed seem like a natural next piece. Once you have those, scoring becomes straightforward.

Do you want to start implementing the checker functions, or tackle something else first?

In [ ]:
values = state.dice_values
# has ones
1 in values
# n in values

from collections import Counter

Counter(values)

Counter({1: 1, 2: 1, 3: 1, 4: 1, 5: 1})

How can I use counter to get keys that have certain value?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

You can filter the Counter by checking the counts:

```python
counts = Counter(values)
# Keys with count >= 2
pairs = [k for k, v in counts.items() if v >= 2]
```

Or to check if any value has a certain count:
```python
has_pair = any(count >= 2 for count in counts.values())
```

Does this help you think about how to implement your checker functions?

In [ ]:
counts = Counter((1, 1, 2, 2, 3))
# Keys with count >= 2
pairs = [k for k, v in counts.items() if v >= 2]
pairs

[1, 2]

In [ ]:
counts = Counter((1, 2, 2, 2, 3))
three_sames = [k for k, v in counts.items() if v >= 3]
three_sames

[2]

In [ ]:
counts = Counter((1, 5, 5, 5, 5))
four_sames = [k for k, v in counts.items() if v >= 4]
four_sames

[5]

In [ ]:
counts = Counter((6,6,6,6,6))
yatzy = [k for k, v in counts.items() if v >= 5]
yatzy

[6]

In [ ]:
# Full house
counts = Counter((5,5,5,6,6))
pairs = [k for k, v in counts.items() if v >= 2]
three_sames = [k for k, v in counts.items() if v >= 3]

pairs.remove(three_sames[0])
len(pairs) > 0

True

In [ ]:
# small straight
values = (1, 2, 3, 4, 5)
values == (1, 2, 3, 4, 5)

# large straight
values = (2, 3, 4, 5, 6)
values == (2, 3, 4, 5, 6)


True

In [ ]:
def has_ns(n, values):
    return n in values

assert has_ns(3, (1,2,3,4,5))
assert not has_ns(6, (1,2,3,4,5))

def n_same(n, values):
    counts = Counter(values)
    return [k for k, v in counts.items() if v >= n]

assert n_same(2, (1,2,2,3,3)) == [2, 3]

def has_n_same(n, values):
    return bool(n_same(n, values))
#def has_pair(values):
#    return n_same(values, 2)

#def has_three_same(values):
#    return n_same(values, 3)

#def has_four_same(values):
#    return n_same(values, 4)

#def has_yatzy(values):
#    return n_same(values, 5)

#assert has_yatzy((1,1,1,1,1))
#assert not has_yatzy((1,1,1,1,2))

def has_two_pairs(values):
    pairs = n_same(2, values)
    if len(pairs) == 2:
        return True
    return False

assert has_two_pairs((1,1,2,2,3))
assert not has_two_pairs((1,1,2,3,4))
assert not has_two_pairs((1,1,1,1,2))

def has_full_house(values):
    pairs = n_same(2, values)
    three_sames = n_same(3, values)
    if pairs and three_sames:
        pairs.remove(three_sames[0])
        return len(pairs) > 0
    return False

assert has_full_house((2,2,3,3,3))
assert not has_full_house((1,2,3,3,3))

def has_small_straight(values):
    return values == (1,2,3,4,5)

def has_large_straight(values):
    return values == (2,3,4,5,6)

In [ ]:
def score_n(value, values): 
    n = values.count(value)
    return value * n
assert score_n(1, (1,1,1,2,3)) == 3

def score_same(n, values): 
    # todo
    return value * n


def score_yatzy(value): return value * 5 + 50
def score_two_pairs(pair1, pair2): return 2 * pair1 + 2 * pair2
def score_full_house(pair_value, three_value): return 2 * pair_value + 3 * three_value
def score_small_straight(): return 15
def score_large_straight(): return 20


In [ ]:
can_zero

{'fives',
 'four_same',
 'fours',
 'full_house',
 'one_pair',
 'ones',
 'sixes',
 'small_straight',
 'three_same',
 'threes',
 'two_pairs',
 'twos'}

In [ ]:
from functools import partial

checkers = {
    "ones": partial(has_ns, 1),
    "twos": partial(has_ns, 2),
    "threes": partial(has_ns, 3),
    "fours": partial(has_ns, 4),
    "fives": partial(has_ns, 5),
    "sixes": partial(has_ns, 6),
    "one_pair": partial(has_n_same, 2),
    "two_pairs": has_two_pairs,
    "three_same": partial(has_n_same, 3),
    "four_same": partial(has_n_same, 4),
    "small_straight": has_small_straight,
    "large_straight": has_large_straight,
    "full_house": has_full_house,
    "chance": lambda x: x, # ???
    "yatzy": partial(n_same, 5),
}

In [ ]:
for category in can_zero:
    can_select = checkers[category](state.dice_values)
    print(category, can_select)

two_pairs False
threes True
small_straight True
fours True
four_same False
one_pair False
twos True
ones True
sixes False
three_same False
full_house False
fives True


In [ ]:
scorers = {
    "ones": partial(score_n, 1),
    "twos": partial(score_n, 2),
    "threes": partial(score_n, 3),
    "fours": partial(score_n, 4),
    "fives": partial(score_n, 5),
    "sixes": partial(score_n, 6),
    "one_pair": partial(score_same, 2),
    "two_pairs": score_two_pairs,
    "three_same": partial(score_same, 3),
    "four_same": partial(score_same, 4),
    "small_straight": score_small_straight,
    "large_straight": score_large_straight,
    "full_house": score_full_house,
    "chance": lambda x: sum(x),
    "yatzy": score_yatzy,
}

In [ ]:
available_actions

[Action(type='reroll', value=(0, 0, 0, 0, 0)),
 Action(type='reroll', value=(0, 0, 0, 0, 1)),
 Action(type='reroll', value=(0, 0, 0, 1, 1)),
 Action(type='reroll', value=(0, 0, 1, 1, 1)),
 Action(type='reroll', value=(0, 1, 1, 1, 1)),
 Action(type='reroll', value=(1, 1, 1, 1, 1))]

In [ ]:
for cat in can_zero:
    available_actions.append(Action(type="zero", value=cat))
    can_select = checkers[cat](state.dice_values)
    if can_select:
        available_actions.append(Action(type="score", value=cat))
available_actions

[Action(type='reroll', value=(0, 0, 0, 0, 0)),
 Action(type='reroll', value=(0, 0, 0, 0, 1)),
 Action(type='reroll', value=(0, 0, 0, 1, 1)),
 Action(type='reroll', value=(0, 0, 1, 1, 1)),
 Action(type='reroll', value=(0, 1, 1, 1, 1)),
 Action(type='reroll', value=(1, 1, 1, 1, 1)),
 Action(type='zero', value='two_pairs'),
 Action(type='zero', value='threes'),
 Action(type='score', value='threes'),
 Action(type='zero', value='small_straight'),
 Action(type='score', value='small_straight'),
 Action(type='zero', value='fours'),
 Action(type='score', value='fours'),
 Action(type='zero', value='four_same'),
 Action(type='zero', value='one_pair'),
 Action(type='zero', value='twos'),
 Action(type='score', value='twos'),
 Action(type='zero', value='ones'),
 Action(type='score', value='ones'),
 Action(type='zero', value='sixes'),
 Action(type='zero', value='three_same'),
 Action(type='zero', value='full_house'),
 Action(type='zero', value='fives'),
 Action(type='score', value='fives')]

Thats all actions I think for $`state`. I need to stop and gather all my work together. I'm interested in Jeremy Howards nbdev approach. He keeps code in notebooks as literate programming. Basically coming up with the solutions is part of that, but the notebooks he has are very clean and straightforward. I guess he cleans them up thoroughly. How should I approach cleaning up this notebook to practice nbdev/solveit approach?